# Using the new found cleaning and feature engineering to experiment further with stacking

## 1 - Any bagging

In [ ]:
scores = []
csv_paths = []
submission_path = "ensemble_predictions/big_bagging_1"
csv_bagging(scores, csv_paths, submission_path)

## 2 - Stacking

In [3]:
%run ../common_utils.py
train, test, metaData = load_all_data()

### XGB advisor 1 😎: all features - predicting floor price

In [4]:
features =           ["building_id", # For grouping
                      "area_total", "area_kitchen", "area_living", "floor", "ceiling", "stories", "rooms",
                      "bathrooms_private", "bathrooms_shared", "balconies","loggias", "phones", "latitude", "longitude", "constructed", # Numerical
                     "layout", "condition", "district", "material", "parking", "heating", "seller", #Categorical
                      "windows_court", "windows_street", "new", "elevator_without", "elevator_passenger", "elevator_service", "garbage_chute", # Bool
                     "street"] #, "address"] # Strings

all_numerical_features = ["area_total", "area_kitchen", "area_living", "floor",
                      "ceiling", "stories", "rooms", "bathrooms_private", "bathrooms_shared", "balconies","loggias", "phones", "latitude", "longitude", "constructed"]

float_numerical_features = ["area_total", "area_kitchen", "area_living", "ceiling", "latitude", "longitude", "constructed"]
int_numerical_features = ["floor", "stories", "rooms", "bathrooms_private", "bathrooms_shared", "balconies", "loggias", "phones"] # Ordinal categories

cat_features = ["layout", "condition", "district", "material", "parking", "heating", "seller"] # All are non-ordinal

droptable = ["street"]

# Load data
train, test, metaData = load_all_data()
# Clean data. 
train_labels, train_targets, test_labels = clean_data(train, test, features, float_numerical_features, int_numerical_features, cat_features,
                                                      log_targets=False, log_area=True, fillNan=False)
# Add new features
train_labels, test_labels, added_features = feature_engineering(train_labels, test_labels,
    add_base_features=True, 
    add_bool_features=True,
    add_weak_features=True,
    add_dist_to_metro=True,
    add_close_to_uni=True,
    add_dist_to_hospital=True,
    add_floor_features=True,
    add_street_info=True,
    add_some_more_features=False                                                            
    )
# Normalize
train_labels, test_labels = normalize(train_labels, test_labels, float_numerical_features, scaler="minMax")
# Drop
train_labels.drop(droptable, inplace=True, axis=1)
test_labels.drop(droptable, inplace=True, axis=1)

minMax


In [5]:
price_per_square_meter = train_targets/train['area_total'] # real price / real_area

In [6]:
xgb_advisor_1 = xgboost.XGBRegressor(max_depth=5, min_child_weight=12, gamma=0.15,
                                 subsample=0.8, colsample_bytree=0.8, reg_alpha=1.1,
                                 reg_lambda=0.3, learning_rate=0.01, n_estimators=10000)

xgb_oof_train_1, xgb_oof_test_1, scores_1 = get_oof_xgboost(xgb_advisor_1, train_labels.drop(['area_total'],axis=1),
                                                      np.log(price_per_square_meter), test_labels.drop(['area_total'],axis=1))

In [7]:
print(scores_1)

[0.20252494570910018, 0.1719402910622423, 0.19194384452156768, 0.18187339676857273, 0.23624429647388234]


### XGB advisor 2 😎: Even more features - as well as area total - predicting price

In [8]:
features =           ["building_id", # For grouping
                      "area_total", "area_kitchen", "area_living", "floor", "ceiling", "stories", "rooms",
                      "bathrooms_private", "bathrooms_shared", "balconies","loggias", "phones", "latitude", "longitude", "constructed", # Numerical
                     "layout", "condition", "district", "material", "parking", "heating", "seller", #Categorical
                      "windows_court", "windows_street", "new", "elevator_without", "elevator_passenger", "elevator_service", "garbage_chute", # Bool
                     "street"] #, "address"] # Strings

all_numerical_features = ["area_total", "area_kitchen", "area_living", "floor",
                      "ceiling", "stories", "rooms", "bathrooms_private", "bathrooms_shared", "balconies","loggias", "phones", "latitude", "longitude", "constructed"]

float_numerical_features = ["area_total", "area_kitchen", "area_living", "ceiling", "latitude", "longitude", "constructed"]
int_numerical_features = ["floor", "stories", "rooms", "bathrooms_private", "bathrooms_shared", "balconies", "loggias", "phones"] # Ordinal categories

cat_features = ["layout", "condition", "district", "material", "parking", "heating", "seller"] # All are non-ordinal

droptable = ["street"]

# Load data
train, test, metaData = load_all_data()
# Clean data. 
train_labels, train_targets, test_labels = clean_data(train, test, features, float_numerical_features, int_numerical_features, cat_features,
                                                      log_targets=True, log_area=True, fillNan=True)
# Add new features
train_labels, test_labels, added_features = feature_engineering(train_labels, test_labels,
    add_base_features=True, 
    add_bool_features=True,
    add_weak_features=True,
    add_dist_to_metro=True,
    add_close_to_uni=True,
    add_dist_to_hospital=True,
    add_floor_features=True,
    add_street_info=True,
    add_some_more_features=True                                                            
    )
# Normalize
train_labels, test_labels = normalize(train_labels, test_labels, float_numerical_features, scaler="minMax")
# Drop
train_labels.drop(droptable, inplace=True, axis=1)
test_labels.drop(droptable, inplace=True, axis=1)

C:\School\høst 2021\TDT4173-ML\common_utils.py:1467: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_labels = train_labels.fillna(train_labels.median()) # Boolean
C:\School\høst 2021\TDT4173-ML\common_utils.py:1476: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test_labels = test_labels.fillna(test_labels.median()) # Boolean


minMax


In [9]:
xgb_advisor_2 = xgboost.XGBRegressor(max_depth=5, min_child_weight=12, gamma=0.15,
                                 subsample=0.8, colsample_bytree=0.8, reg_alpha=1.1,
                                 reg_lambda=0.3, learning_rate=0.01, n_estimators=10000)

xgb_oof_train_2, xgb_oof_test_2, scores_2 = get_oof_xgboost(xgb_advisor_2, train_labels,
                                                      train_targets, test_labels)

In [10]:
print(scores_2)

[0.21366608257200162, 0.17480404267524746, 0.19900553175201274, 0.18699747945287365, 0.2390275085244918]


### DL advisor 1 - Predicting area_price

In [61]:
%run ../common_utils.py

In [62]:
# Mostly the ones correlated to price.
float_numerical_features = ["area_total", "area_kitchen", "area_living", "latitude", "longitude", "ceiling"]

int_numerical_features = ["floor", "stories", "rooms", "bathrooms_shared", "bathrooms_private", "balconies", "constructed"]

cat_features = ["district", "material", "parking", "seller"]

bool_features = []

features = ["building_id", "street"] + float_numerical_features + int_numerical_features + cat_features + bool_features

# Many of the features was just added to help clean/engineer other features.
droptable = ['longitude', 'latitude', 'area_kitchen', 'area_living', 'floor', 'stories', "street",
            "rel_living", "bathrooms_shared", "bathrooms_private", "theta"] 

In [63]:
# Load data
train, test, metaData = load_all_data()

In [64]:
# Clean data
train_labels, train_targets, test_labels = clean_data(train, test, features, float_numerical_features, int_numerical_features, cat_features,
                                                      log_targets=False, log_area=False, fillNan=True)

C:\School\høst 2021\TDT4173-ML\common_utils.py:1468: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_labels = train_labels.fillna(train_labels.median()) # Boolean
C:\School\høst 2021\TDT4173-ML\common_utils.py:1477: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test_labels = test_labels.fillna(test_labels.median()) # Boolean


In [65]:
# Feature engineering
train_labels, test_labels, added_features = feature_engineering(train_labels, test_labels,
    add_base_features=True, 
    add_bool_features=False,
    add_weak_features=False,
    add_dist_to_metro=True,
    add_close_to_uni=False,
    add_dist_to_hospital=False,
    add_floor_features=False,
    add_street_info=True,
    )

In [66]:
# Normalize
train_labels, test_labels = normalize(train_labels, test_labels, float_numerical_features, scaler="minMax")

minMax


In [67]:
# One-hot encoding
train_labels, test_labels = one_hot_encoder(train_labels, test_labels,
                                            cat_features, drop_old=True)

In [68]:
# Droptable
train_labels.drop(droptable, inplace=True, axis=1)
test_labels.drop(droptable, inplace=True, axis=1)

In [69]:
price_per_square_meter = train_targets/train['area_total']

In [ ]:
model_params = ([18, 18, 18], tensorflow.nn.leaky_relu,
                     [False, False, False], 0.2, 'adam',
                      rmsle_custom, ['mse', 'msle', tensorflow.keras.metrics.Accuracy()], True)

ann_oof_train_1, ann_oof_test_1, hists_1 = get_oof_ann(model_params,
                                                 train_labels.drop(['area_total'],axis=1),
                                                 price_per_square_meter,
                                                 test_labels.drop(['area_total'],axis=1))


....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
......................................................................Epoch 00570: early stopping

....................................................................................................
....................................................................................................
....................................................................................................
............................................................................................

In [ ]:
for history in hists_1:
    print(history.history["val_loss"][-1], end=", ")

### DL advisor 2 - Predicting price

In [ ]:
# Mostly the ones correlated to price.
float_numerical_features = ["area_total", "area_kitchen", "area_living", "latitude", "longitude", "ceiling"]

int_numerical_features = ["floor", "stories", "rooms", "bathrooms_shared", "bathrooms_private", "balconies", "constructed"]

cat_features = ["district", "material", "parking", "seller"]

bool_features = []

features = ["building_id", "street"] + float_numerical_features + int_numerical_features + cat_features + bool_features

# Many of the features was just added to help clean/engineer other features.
droptable = ['longitude', 'latitude', 'area_kitchen', 'area_living', 'floor', 'stories', "street",
            "rel_living", "bathrooms_shared", "bathrooms_private", "theta"]

In [ ]:
# Load data
train, test, metaData = load_all_data()

In [ ]:
# Clean data
train_labels, train_targets, test_labels = clean_data(train, test, features, float_numerical_features, int_numerical_features, cat_features,
                                                      log_targets=False, log_area=False, fillNan=True)

In [ ]:
# Feature engineering
train_labels, test_labels, added_features = feature_engineering(train_labels, test_labels,
    add_base_features=True, 
    add_bool_features=False,
    add_weak_features=False,
    add_dist_to_metro=True,
    add_close_to_uni=False,
    add_dist_to_hospital=False,
    add_floor_features=False,
    add_street_info=True,
    )

In [ ]:
# Normalize
train_labels, test_labels = normalize(train_labels, test_labels, float_numerical_features, scaler="minMax")

In [ ]:
# Droptable
train_labels.drop(droptable, inplace=True, axis=1)
test_labels.drop(droptable, inplace=True, axis=1)

In [ ]:
model_params = ([18, 18, 18], tensorflow.nn.leaky_relu,
                     [False, False, False], 0.2, 'adam',
                      rmsle_custom, ['mse', 'msle', tensorflow.keras.metrics.Accuracy()], True)

ann_oof_train_2, ann_oof_test_2, hists_2 = get_oof_ann(model_params,
                                                 train_labels,
                                                 train_targets,
                                                 test_labels)

In [ ]:
for history in hists_2:
    print(history.history["val_loss"][-1], end=", ")